In [1]:
import pandas as pd
import numpy as np

In [2]:
# code for reading all HTLM files in the directory

In [3]:
import os, sys

wp_path = "/Users/ginevralarroux/Desktop/EPFL courses/Applied data analysis/ADA project/data/wpcd/wp/" # to adapt
directories = os.listdir(path)

html_list = []
for dir in directories:
    local_path = path + dir
    local_dir = os.listdir( local_path )
    for link in local_dir:
        if link[-3:]=='htm':
            html_list.append(dir+'/'+link)
        
html_list

NameError: name 'path' is not defined

The aim of this specific game strategy analysis is to test whether Wikispeedia players have clikability preferences on the positioning of the hyperlinks in the article text. Firstly we will address their postioning in the text, evaluated on the basis of the ordinal number of the paragraph they show up in; secondly we will test if there is a correlation between the clickthrough rate of the hyperlinks and their location in image captions. 

It is therefore essential to extract from the data the clicktrhough rate for each hyperlink.

In [ ]:
# code for extracting the clickthrough rate of hyperlinks

In [ ]:
folder='/Users/ginevralarroux/Desktop/EPFL courses/Applied data analysis/ADA project/data/wikispeedia_paths-and-graph/'

paths_finished='paths_finished.tsv'
articles='articles.tsv'

paths_finished=pd.read_csv(folder+paths_finished, sep='\t', skiprows=15, header=None, names=['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'rating'])
articles=pd.read_csv(folder+articles, sep='\t', skiprows=12, header=None, names=['article'])

This dataframe contains the paths chosen by the players who could reach the target article.

In [ ]:
paths_finished.head()

This dataframe lists all the clickable articles.

In [ ]:
def filter_article_name(article_column):
    article_column=article_column.str.lower()
    article_column=article_column.str.replace('_', ' ')
    return article_column

articles.article=filter_article_name(articles.article)
articles.head()

In [ ]:
# this is the list articles in the lists of paths
    
clicked_articles=[]
for x in paths_finished['path']: # hypothesize also in paths_unfinished
    path=(x.split(';'))
    clicked_articles.append(path)

#for path in clicked_articles:
#    path_series=pd.Series(path)
#    clicked_articles=filter_article_name(path_series)

clicked_articles

In [ ]:
# this is the dataframe containing the hyperlinks and their clicktrhough rates

links_name=[]
links_freq=[]

i=0
for article in articles['article']:
    count=0
    links_name.append(article)
    for i in range(len(clicked_links)):
        if article in clicked_links[i]:
            count+=1 
    links_freq.append(count)

links_name=pd.Series(links_name)
links_freq=pd.Series(links_freq)
links_freq_df=pd.concat([links_name, links_freq], axis=1, names=['link', '#_clicks'])
links_freq_df.columns=['link', '#_clicks']
links_freq_df.head(20)

In [ ]:
# code for extracting the positioning of the hyperlinks in the article text

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
URL = '/Users/ginevralarroux/Desktop/EPFL courses/Applied data analysis/ADA project/data/wpcd/wp/a/A_cappella.htm'

f=open(URL, 'r')
soup = BeautifulSoup(f, 'lxml')

In [ ]:
# read all the files
soups=[]
for html in html_list:
    f=open(wp_path+html, 'r')
    soup = BeautifulSoup(f, 'lxml')
    soups.append(soup)
    
soups

In [ ]:
paragraphs = soup.find_all('p')
print('The total number of paragraphs is {0}.'.format(len(paragraphs)))

In [ ]:
# HTML tag for hyperlinks is <a href= >
# this is the dataframe of the hyperlinks contained in an article, with the number of paragraph they show up in 
# and its relative postion in the article

paragraphs = soup.find_all('p')

links=[]
i=0

for p in paragraphs:
    i+=1
    all_links=p.find_all('a')
    for link in all_links:
        if 'href' in link.attrs:
            link_title=link.text
            link_paragraph=i
            links.append([link_title, link_paragraph])

link_positioning_df=pd.DataFrame(links, columns=['link', '#_paragraph']).drop_duplicates()
link_positioning_df['positioning']=round(link_positioning_df['#_paragraph']/len(paragraphs), 2)
link_positioning_df

In [ ]:
# code for finding hyperlinks in images' caption

In [ ]:
URL = '/Users/ginevralarroux/Desktop/EPFL courses/Applied data analysis/ADA project/data/wpcd/wp/a/Aachen.htm'
f=open(URL, 'r')
soup2 = BeautifulSoup(f, 'lxml')

In [ ]:
# in an HTML file the images can be either find under tables or div tags 

In [ ]:
tables = soup2.find_all('table')
print('The total number of tables is {0}.'.format(len(tables)))

In [ ]:
img = soup2.find_all('img')
print('The total number of images is {0}.'.format(len(img)))

In [ ]:
# images under table tags
# dataframe of hyperlinks in tables' captions

links=[]

for table in tables:
    if (table.img):
        all_links= table.find_all('a')
        for link in all_links:
            if 'class' not in link.attrs:
                link_title=link.text
                links.append(link_title)
        
links_in_table_df=pd.DataFrame(links)
links_in_table_df.columns=['link_in_table']
links_in_table_df

In [ ]:
# images under div tags
# dataframe of hyperlinks in tables' captions

div = soup2.find_all('div')

links=[]

for d in div:
    if (d.find('a', class_="internal")):
        if (d.find('img', class_='thumbimage')):
            caption=d.find('div', class_="thumbcaption")
            if (caption.find('a')):
                all_links=caption.find_all('a')
                for link in all_links:
                    if 'class' not in link.attrs:
                        link_title=link.text
                        links.append(link_title)
                            
links_in_image_df=pd.DataFrame(links).drop_duplicates() 
links_in_image_df.columns=['link_in_image']
links_in_image_df    

In [ ]:
# when we'll have all the articles' dataframes we will merge them by link and get the average of their positioning
# we will sort the articles based on their clickthrough rate, ascending=False
# we will check if there is a correlation between the positioning and clickthrough rate

In [ ]:
# when we'll have all the articles' dataframes we will label with a flag the hyperlinks located in images' captions
# we will check if there is a correlation between the location in an image caption and clickthrough rate

In [ ]:
# assumptions: 
# we don't know for the hyperlinks that show up both in the article text and in the image caption, 
# which the player actually clicked 
# for the hyperlinks which show up multiple times in the article ...


In [ ]:
URL = '/Users/ginevralarroux/Desktop/EPFL courses/Applied data analysis/ADA project/data/wpcd/wp/m/Music.htm'

f_music=open(URL, 'r')
soup_music = BeautifulSoup(f_music, 'lxml')

In [ ]:
paragraphs_m = soup_music.find_all('p')
print('The total number of paragraphs is {0}.'.format(len(paragraphs_m)))

links_m=[]
i=0

for p in paragraphs_m:
    i+=1
    all_links=p.find_all('a')
    for link in all_links:
        if 'href' in link.attrs:
            link_title=link.text
            link_paragraph=i
            links_m.append([link_title, link_paragraph])

link_positioning_df_m=pd.DataFrame(links, columns=['link', '#_paragraph']).drop_duplicates()
link_positioning_df_m['positioning']=round(link_positioning_df_m['#_paragraph']/len(paragraphs_m), 2)
link_positioning_df_m

In [ ]:
merged=link_positioning_df.merge(link_positioning_df_m, how='outer', left_on='link', right_on='link', )
columns_to_drop=['#_paragraph_x', '#_paragraph_y']
m=merged.drop(labels=columns_to_drop, axis=1)
print(len(m))
m.reset_index(drop=True, inplace=True)
m

In [ ]:
mean_positioning=m.groupby(by=m['link']).apply(np.mean, axis=1)
mean_positioning_df=pd.DataFrame(mean_positioning, columns=['mean_positioning'])
mean_positioning_df.reset_index(level=1, drop=True, inplace=True)
mean_positioning_df=mean_positioning_df.reset_index()
mean_positioning_df
#mean_positioning_df['link'].unique()

In [ ]:
a=links_freq_df

new_data=a.merge(mean_positioning_df, left_on='link', right_on='link')
new_data

In [ ]:
#function to better take into account than the mean 